<a id='1'></a>
<div class="alert alert-block alert-danger">
<h2>1 Import Packages</h2>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # ignore information messages

import tensorflow as tf
from tensorflow.keras.datasets import mnist
# images of digits between 0 and 9 with 60000 train and 10000 test grayscale 28x28 pixels images
from tensorflow.keras import Sequential, Input, Model, regularizers
from tensorflow.keras.layers import (Layer, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, 
                                     GlobalAveragePooling2D)
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu

import pandas as pd

Init Plugin
Init Graph Optimizer
Init Kernel


<a id='2'></a>
<div class="alert alert-block alert-danger">
   <h2>
    2 Load Dataset
    </h2>
</div>

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

<a id='3'></a>
<div class="alert alert-block alert-danger">
   <h2>
    3 Create a neural network
    </h2>
</div>

In [3]:
# class Dense(Layer):
#     def __init__(self, units, input_dim):
#         super(Dense, self).__init__()
#         self.w = self.add_weight(
#             name="w",
#             shape=(input_dim, units),
#             initializer="random_normal",
#             trainable=True,
#         )
#         self.b = self.add_weight(
#             name="b", shape=(units,), initializer="zeros", trainable=True
#         )

#     def call(self, inputs):
#         return tf.matmul(inputs, self.w) + self.b


class Dense(Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b", shape=(self.units,), initializer="random_normal", trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


class MyReLU(Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)


class MyModel(Model):  # model.fit, model.evalute, model.predict
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()

        # self.dense1 = layers.Dense(64)
        # self.dense3 = layers.Dense(num_classes)

    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

In [4]:
model = MyModel()
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(),
    metrics=["accuracy"],
)

Metal device set to: Apple M1


In [5]:
model.fit(x_train, y_train, batch_size=32, epochs=2)
model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3409 - accuracy: 0.9059
Epoch 2/2
313/313 [==============================] - 1s 3ms/step - loss: 0.1337 - accuracy: 0.9605


[0.1336686909198761, 0.9605000615119934]